# Host Web App

There are many ways to host a web app, here we'll simply run EC2 instance and host the web app on it. We'll use Streamlit for it.

Here is a similar [blog post](https://towardsdatascience.com/how-to-deploy-a-streamlit-app-using-an-amazon-free-ec2-instance-416a41f69dc3) which explains the process in detail.

![](assets/system.jpg)
## Create an instance

This should be straightforward: 
- AMI: I use ubuntu since it is easier.
- Security group: create a new one with two inbound rules:
    - SSH on port 22 and 
    - Custom TCP on port 8501, this is Streamlit default port. Any new app will run on 8502, etc. so make sure to add those to rules as well.
    - HTTP at port 80, if you want to host a website.
- Key-pair: create a new one and download it, say it's name is `streamlit.pam`. This is the only time you can download it, so make sure to keep it safe.



## Connect to the instance 

Connect to the instance using the following command. Replace `<Your Public DNS(IPv4) Address>` with your instance's public DNS address.
```zsh
chmod 400 streamlit.pem
ssh -i "streamlit.pem" ubuntu@<Your Public DNS(IPv4) Address>
```

Now we install miniconda:
```zsh
sudo apt-get update
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
bash ~/miniconda.sh -b -p ~/miniconda
echo "PATH=$PATH:$HOME/miniconda/bin" >> ~/.bashrc
source ~/.bashrc
```

Get the code from our repository and install the requirements.
```zsh
git clone XYZ
pip install -r requirements.txt
```



## Tmux

```zsh
sudo apt-get install tmux
tmux new -s StreamSession  # create a new session called StreamSession
tmux list-sessions  # to discover sessions
tmux attach -t StreamSession  # to reatach
tmux kill-session -t StreamSession  # to kill session
```

To leave tmux session, press `Ctrl+b` and then `d`.



## Add DNS name 

One option is to add subdomain, for example mygpt.nenadbozinovic.com will now go to IP address of the EC2 instance. The port will be 80 by default.

![dns](assets/DNS_setup2.jpg)




## Prep for SendCommand remotely

First we need to assume user role, which is done by specifying AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY. And we need to make sure this user has the necessary permissions to send command, by attaching SendCommand policy to the user.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "ssm:SendCommand",
            "Resource": "*"
        }
    ]
}
```

Note that we don't care what role instance has only what role user who is trying to access instance has.

We also need to make sure instance can accept commands, this should be default and done by snap, double check it is installed, enabled, and active:
```zsh
snap services amazon-ssm-agent
```

We need to run streamlit so we need to execute these commands:
```zsh
cd /home/ubuntu/blog && git pull && cd nbs/projects/myGPT && streamlit run myGPT.py --server.enableCORS false --server.enableXsrfProtection false
```

Note that `--server.enableCORS false --server.enableXsrfProtection false` reduces security but allows app to run on servers. One can get away running without the flags too.

This should have myGPT running when executing on a remote machine.

## SendCommand

To run command remotely, we need a few more things:
- we need `nohup` call so the process can run in the background
- we need to specify path to streamlit, since it is not in the PATH
- we should also redirect output to a file so we can see what is happening

```zsh
cd /home/ubuntu/blog && git pull && cd nbs/projects/myGPT && nohup /home/ubuntu/miniconda/bin/streamlit run myGPT.py --server.enableCORS false --server.enableXsrfProtection false >> output.log 2>&1
```

The following allows to append (>>) to output.log and redirect stderr to stdout (2>&1):
```zsh
>> output.log 2>&1
```

So let's do it then, we execute command remotely using AWS Systems Manager (or `aws ssm`):
```zsh
aws ssm send-command \
    --document-name "AWS-RunShellScript" \
    --targets "Key=instanceids,Values=i-08b8b6691ed2e1b6d" \
    --parameters commands="cd /home/ubuntu/blog && git pull && cd nbs/projects/myGPT && nohup /home/ubuntu/miniconda/bin/streamlit run myGPT.py --server.enableCORS false --server.enableXsrfProtection false >> output.log 2>&1" \
    --region us-west-1
```

We can now go to the remote machine and see what is running on the instance (and or kill it):
```zsh
ps aux | grep streamlit
kill <PID>
```


To run app:
```zsh
streamlit run main.py --server.enableCORS false --server.enableXsrfProtection false
```